<a href="https://colab.research.google.com/github/vaylonn/reconnaissance-faciale/blob/main/reconnaissance_faciale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Pour permettre a collab d'acceder aux images dans le drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          image: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  image = cv2.imdecode(jpg_as_np, flags=1)

  return image

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'image': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data



In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
#face_recgnition n'est pas installé de base sur colab
!pip install face_recognition

In [ ]:
#on importe les bibliothèques

import numpy as np
import cv2 #bibli super pour le traitement des images
import matplotlib.pyplot as plt #on utilise plt pou afficher les images car la fonction imshow de cv2 est desactivée dans colab a cause de bugs
import face_recognition #bibli qui contiens pleins d'algos pour la reconnaissance faciale
import os #pour naviguer dans les fichiers

Recuperer les images

In [ ]:
#on creer les listes qui vont contenirs nos données
known_faces_encoding = []
known_faces_names = []

path = '/content/drive/MyDrive/visages/' #toutes les images on ça comme début de chemin

faces =  os.listdir(path) #permet de recuperer les noms de tous les fichiers qu'il y a dans le dossier

for face in faces:
  face_path = path+face #le chemin vers l'image est le chemin vers le dossier + le nom de l'image

  # Lire les images
  # ...
  image = face_recognition.load_image_file(face_path)
  # Convertir les images en gris 
  # ...

  # Encoder les visages avec la lib face recognition et l'ajouter dans la liste
  # ...
  image_encoding = face_recognition.face_encodings(image)[0]
  # Recuperer le nom avec os et l'ajouter dans la liste
  # ...
  known_faces_encoding.append(image_encoding)
#path = chemin, basename = recup le nom d'un fichier dans le chemin
  name = os.path.basename(face_path).replace(".jpg","").replace(".JPG", "").replace(".png","")
  known_faces_names.append(name)

In [ ]:
video_stream()
label_html = 'Capturing...'
bbox = ''
count = 0 
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break
    frame = js_to_image(js_reply["image"])
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)




    # creation d'une boite de pixels transparents devant l'image
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # Detecter les visages dans l'image
    # ...
    face_locations = face_recognition.face_locations(frame)
    # Encoder les visages
    # ...
    face_encodings = face_recognition.face_encodings(frame, face_locations)
    #on prépare une liste pour mettre les noms des visages détectés
    face_names = [] 

    # boucle pour detecter et comparer tous les visages de l'image
    for face_encoding in face_encodings:
      # on regarde si le visage ressemble a un visage connu avec la lib face recognition
      matches = face_recognition.compare_faces(known_faces_encoding, face_encoding)
      name = "Inconnu"

      # Calcul de la distance entre le visage trouvé et les visages connus et on l'ajoute dans la liste
      face_distances = face_recognition.face_distance(known_faces_encoding, face_encoding)
      best_match_index = np.argmin(face_distances)
      if matches[best_match_index]:
        name = known_faces_names[best_match_index]
      face_names.append(name)

    # Boucle pour afficher les visages reconnus
    # ...
    for i in range (len(face_locations)):
      locations = face_locations[i]
      name = face_names[i]

      top, left, bottom, right = locations[0], locations[1], locations[2], locations[3]

      # Tracez un rectangle autour du visage et on mettez le nom (modifiez la bbox)
      bbox_array = cv2.rectangle(bbox_array, (left, top), (right, bottom), (200,0,0), 2)
      bbox_array = cv2.putText(bbox_array, name, (left, top-10), cv2.FONT_HERSHEY_COMPLEX, 1,(200,0,0))



    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    bbox_bytes = bbox_to_bytes(bbox_array)
    bbox = bbox_bytes